In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import folium
import shapely
import pyproj
from shapely.ops import transform

## nyc_gdf cleaning

In [3]:
# read data and transform to wgs84

nyc_gdf = gpd.read_file('nycsquirrels_clean_1.csv')
nyc_gdf = gpd.GeoDataFrame(nyc_gdf, geometry=gpd.points_from_xy(nyc_gdf.long, nyc_gdf.lat))
nyc_gdf1 = nyc_gdf.set_crs('epsg:4326')

In [4]:
# convert fake bools into 0s and 1s

long_float = nyc_gdf1['long'].apply(lambda x: float(x))
lat_float = nyc_gdf['lat'].apply(lambda x: float(x))

bool_to_int = lambda x: 1 if x == 'TRUE' else 0

approaches_int = nyc_gdf1['approaches'].apply(bool_to_int)
indifferent_int = nyc_gdf1['indifferent'].apply(bool_to_int)
runs_from_int = nyc_gdf1['runs_from'].apply(bool_to_int)

running_int = nyc_gdf1['running'].apply(bool_to_int)
chasing_int = nyc_gdf1['chasing'].apply(bool_to_int)
climbing_int = nyc_gdf1['climbing'].apply(bool_to_int)
eating_int = nyc_gdf1['eating'].apply(bool_to_int)
foraging_int = nyc_gdf1['foraging'].apply(bool_to_int)

kuks_int = nyc_gdf1['kuks'].apply(bool_to_int)
quaas_int = nyc_gdf1['quaas'].apply(bool_to_int)
moans_int = nyc_gdf1['moans'].apply(bool_to_int)
tail_flags_int = nyc_gdf1['tail_flags'].apply(bool_to_int)
tail_twitches_int = nyc_gdf1['tail_twitches'].apply(bool_to_int)

nyc_gdf2 = nyc_gdf1.assign(long = long_float,
                           lat = lat_float,
                           approaches = approaches_int,
                           indifferent=indifferent_int,
                           runs_from = runs_from_int,
                           running = running_int,
                           chasing = chasing_int,
                           climbing = climbing_int,
                           eating = eating_int,
                           foraging = foraging_int,
                           kuks = kuks_int,
                           quaas = quaas_int,
                           moans = moans_int,
                           tail_flags = tail_flags_int,
                           tail_twitches = tail_twitches_int)

## adding data from osm

In [5]:
### load geojson datasets

# water bodies and streams
centralpark_water_osm = gpd.read_file('geojson/water_cp.geojson')
centralpark_stream_osm = gpd.read_file('geojson/streams_cp.geojson')

# playground
centralpark_playg_osm = gpd.read_file('geojson/playgrounds_cp.geojson')

# road (perimeter and inside)
centralpark_paved_osm = gpd.read_file('geojson/paved.geojson')

# toilets (buildings and points)
centralpark_toilets_osm = gpd.read_file('geojson/toilets.geojson')

# buildings (by section because too much data at once)
cp_northbd_osm = gpd.read_file('geojson/northbd.geojson') # cp north
cp_jorbd_osm = gpd.read_file('geojson/jorbd.geojson') # jacqueline kennedy onassis reservoir
cp_greatlawnbd_osm = gpd.read_file('geojson/greatlawnbd.geojson') # great lawn
cp_ramblebd_osm = gpd.read_file('geojson/ramblebd.geojson') # ramble
cp_southbd_osm = gpd.read_file('geojson/southbd.geojson') # cp south
met_osm = gpd.read_file('geojson/met.geojson') # the met

# gardens
centralpark_gardens_osm = gpd.read_file('geojson/gardens_cp.geojson')

# woods
allwoodsosm = gpd.read_file('geojson/woods_cp.geojson')

# baseball pitches (brown, inconsistent with fields/sports center)
cp_baseball_osm = gpd.read_file('geojson/baseballpitch.geojson')

# fields (open grassy areas)
cp_field_osm = gpd.read_file('geojson/field.geojson')

# bare rock (didn't know this was a thing)
allbarerockosm = gpd.read_file('geojson/bare_rock.geojson')

# remaining paved areas (The Mall, etc.)
cp_pedestrian_osm = gpd.read_file('geojson/pedestrian.geojson')

# sports center (tennis courts, recreation center)
cp_sportscenter_osm = gpd.read_file('geojson/sportscenter.geojson')

In [7]:
### data cleaning

# allwaterosm

drop_water_osm = ['relation/2389611', 'relation/2389521', 'relation/2389581', 
                  'relation/2389563','relation/2212963', 'way/608955193', 
                  'way/375673047', 'way/375673046', 'way/1024586392',
                  'way/727185210', 'way/727185216', 'way/727185221', 'way/727185214', 
                  'way/113690622', 'way/163470962', 'way/260910559', 'way/319615314', 
                  'way/319615316', 'way/394395237', 'way/394395238', 'way/470714268', 
                  'way/988718514', 'way/958630356']

drop_water_index = [centralpark_water_osm[centralpark_water_osm.get('id') == 
                                          water_id].index[0] 
                    for water_id in drop_water_osm]

cp_water_osm = centralpark_water_osm.drop(index=drop_water_index)

allwaterosm = pd.concat([cp_water_osm, centralpark_stream_osm])
allwaterosm = allwaterosm.reset_index().drop(columns = 'index')


# allplaygosm

# filling in n/a values
centralpark_playg_osm.at[23, 'name'] = 'Rudin Family Playground'
centralpark_playg_osm.at[50, 'name'] = 'Safari Playground'

cp_playgrounds_name = ['West 110th Street Playground', 'Tarr Family Playground', 
                       'Rudin Family Playground','Robert Bendheim Playground', 
                       'Tarr-Coyne Wild West Playground', '96th Street Playground', 
                       'Abraham and Joseph Spector Playground', 
                       'Arthur Ross Pinetum Playground', 
                       'Diana Ross Playground', 'Ancient Playground', 'Safari Playground',
                       'Ruth and Arthur Smadbeck-Heckscher East Playground', 
                       'Adventure Playground', 'Tarr-Coyne Tots Playground', 
                       'East 72nd Street Playground', 'Billy Johnson Playground', 
                       'Hecksher Playground', 'Mariner\'s Playground']

cp_playgrounds_index = [centralpark_playg_osm[centralpark_playg_osm.get('name') == 
                                              playground_name].index[0]
                        for playground_name in cp_playgrounds_name]

allplaygosm = centralpark_playg_osm.loc[cp_playgrounds_index]


# allpavedosm

drop_paved_name = ['Park Avenue', 'Madison Avenue', 'Lexington Avenue', 
                   'York Avenue', 'FDR Drive', 
                   'Riverside Drive', 'West End Avenue', 
                   'Frederick Douglass Boulevard', 'Malcolm X Boulevard',
                   '8th Avenue', '7th Avenue', '6th Avenue', 
                   'East 106th Street', 'East 97th Street', 'East 96th Street', 
                   'East 86th Street', 'East 85th Street', 'East 84th Street', 
                   'East 79th Street',
                   'East 72nd Street', 'East 66th Street', 'East 65th Street',
                   'East 60th Street', 'East 59th Street',
                   'East 73rd Street', 'East 71st Street', 'East 61st Street',
                   'Cathedral Parkway', 'West 110th Street', 'West 106th Street', 
                   'West 96th Street', 
                   'West 95th Street', 'West 86th Street', 'West 81st Street', 
                   'West 79th Street', 
                   'West 75th Street', 'West 72nd Street', 'West 66th Street', 
                   'West 65th Street',
                   'West 62nd Street', 'West 54th Street', 'West 53rd Street']

drop_paved_index = []

for name in drop_paved_name:
    index_list = centralpark_paved_osm[centralpark_paved_osm.get('name') == name].index
    for index in index_list:
        drop_paved_index.append(index)
        
drop_5th_ends = ['way/988984235', 'way/195743336', 'way/422298615', 
                 'way/962626218', 'way/198924778',
                 'way/840996403', 'way/980855831', 'way/980855832', 
                 'way/980855833', 'way/22898583',
                 'way/46613661', 'way/5670874', 'way/167922075', 
                 'way/584850094', 'way/834058007', 'way/5673363', 'way/584850099']

for way in drop_5th_ends:
    drop_paved_index.append(centralpark_paved_osm[centralpark_paved_osm.get('id') == 
                                                  way].index[0])
    

allpavedosm = centralpark_paved_osm.drop(index = drop_paved_index)


# alltoiletsosm

drop_toilets = ['node/2329911645', 'node/8539548031', 'node/1310874079', 
                'node/2339622631', 'way/266919662']

drop_toilets_index = [centralpark_toilets_osm[centralpark_toilets_osm.get('id') == 
                                              node].index[0]
                      for node in drop_toilets]

alltoiletsosm = centralpark_toilets_osm.drop(index = drop_toilets_index)


# allbuildingosm

keepbd_columns = ['id', 'alt_name', 'amenity', 'building', 'height', 'name', 'geometry']

cp_northbd_id = ['way/39243913', 'way/278366155', 'way/278366170', 'way/118729389', 
                 'way/943419152',
                 'way/278366166', 'way/278366163', 'way/278366174']
cp_northbd_index = [cp_northbd_osm[cp_northbd_osm.get('id') == northbd_id].index[0] 
                    for northbd_id in cp_northbd_id]
allnorthbdosm = cp_northbd_osm.loc[cp_northbd_index].get(keepbd_columns)

cp_jorbd_id = ['way/278363031', 'way/817397167', 'way/278363044']
cp_jorbd_index = [cp_jorbd_osm[cp_jorbd_osm.get('id') == jorbd_id].index[0] 
                  for jorbd_id in cp_jorbd_id]
alljorbdosm = cp_jorbd_osm.loc[cp_jorbd_index].get(keepbd_columns)

cp_greatlawnbd_id = ['way/278363039', 'way/278363038', 'way/278363033', 
                     'way/278363041', 'way/278363029',
                     'way/278363036', 'way/278363065', 'way/278363049', 
                     'way/278363046', 'way/278363024', 
                     'way/278363048', 'way/278363035', 'way/278363053']
cp_greatlawnbd_index = [cp_greatlawnbd_osm[cp_greatlawnbd_osm.get('id') == 
                                           greatlawnbd_id].index[0] 
                        for greatlawnbd_id in cp_greatlawnbd_id]
allgreatlawnbdosm = cp_greatlawnbd_osm.loc[cp_greatlawnbd_index].get(keepbd_columns)

cp_ramblebd_id = ['way/278361406', 'way/278361407', 'way/775820359', 'relation/3698871', 
                  'way/354587581', 'way/354587583', 'way/265347581', 'way/265347585', 
                  'way/362647331', 'way/265347579', 'way/265347587']
cp_ramblebd_index = [cp_ramblebd_osm[cp_ramblebd_osm.get('id') == ramblebd_id].index[0] 
                     for ramblebd_id in cp_ramblebd_id]
allramblebdosm = cp_ramblebd_osm.loc[cp_ramblebd_index].get(keepbd_columns)

cp_southbd_id = ['way/265347589', 'way/585788256', 'way/265347597', 
                 'way/265347588', 'way/265347593',
                 'way/321166650', 'way/321166649', 'way/265347596', 
                 'way/265347590', 'way/265347591',
                 'way/385442739', 'way/265347586', 'way/265347592']
cp_southbd_index = [cp_southbd_osm[cp_southbd_osm.get('id') == southbd_id].index[0] 
                    for southbd_id in cp_southbd_id]
allsouthbdosm = cp_southbd_osm.loc[cp_southbd_index].get(keepbd_columns)

allmetosm = met_osm[met_osm.get('name') == 'The Metropolitan Museum of Art']

allbuildingosm = pd.concat([allnorthbdosm, alljorbdosm, allgreatlawnbdosm, 
                            allramblebdosm, allsouthbdosm, allmetosm])


# allgardenosm

drop_garden = ['way/362826846', 'way/169915675', 'way/330448922', 
               'way/313994014', 'way/202503285',
               'way/225189389', 'way/796333282', 'way/225188744']

drop_garden_index = [centralpark_gardens_osm[centralpark_gardens_osm.get('id') == 
                                             way].index[0] 
                     for way in drop_garden]
    
allgardenosm = centralpark_gardens_osm.drop(index = drop_garden_index)


# allbaseballosm

drop_baseball = ['way/667343443', 'way/920569850', 'way/920569847', 'way/319797239']

drop_baseball_index = [cp_baseball_osm[cp_baseball_osm.get('id')
                                       == way].index[0] for way in drop_baseball]

allbaseballosm = cp_baseball_osm.drop(index = drop_baseball_index)


# allfieldosm

drop_field = ['way/188692199', 'way/951525207', 'way/951525216', 'way/951525219', 
              'way/951539223', 'way/951539207', 'way/951539204', 'way/951539221', 
              'way/951539203', 'way/951539215',
              'way/951539218', 'way/951525211', 'way/951539232', 'way/951525215', 
              'way/951525217',
              'way/951539196', 'way/951525222', 'way/951525209', 'way/972597724',
              'way/949457222', 'way/949457226', 'way/953189819', 'way/849307005', 
              'way/849307011', 'way/849307012',
              'way/849307006', 'way/952333244', 'way/849307009', 'way/953189823', 
              'way/952333240', 'way/949532332',
              'way/949848463', 'way/949851336', 'way/667981482', 'way/949848467', 
              'way/949848466',
              'way/833371440', 'way/949848471', 'way/949848459', 'way/949848457',
              'way/849046450', 'way/849046418', 'way/849046432', 'way/849046434', 
              'way/849046451',
              'way/849046431', 'way/849046424', 'way/849046433', 'way/849046423', 
              'way/667981479', 'way/667981483',
              'way/667981480', 'way/849046437', 'way/849046453', 'way/849046448', 
              'way/849046420',
              'way/849046440', 'way/849046435', 'way/849046417', 'way/849299252', 
              'way/849299251', 'way/949848465',
              'way/849046452', 'way/849046454', 'way/849046415', 'way/667981478', 
              'way/849046438', 'way/849046427',
              'way/849046421', 'way/849046429', 'way/849046457', 'way/849046428',
              'way/849046444', 'way/849046441', 'way/849046426', 'way/849046414', 
              'way/949848462']

drop_field_index = [cp_field_osm[cp_field_osm.get('id') == way].index[0] 
                    for way in drop_field]

allfieldosm = cp_field_osm.drop(index = drop_field_index)


# allpedestrianosm

drop_ped = ['relation/4623434', 'relation/4623433', 'relation/4623431', 
            'way/1031859264', 'way/321166645',
            'relation/2280942', 'way/329988122', 'way/329988123', 
            'way/458452177', 'way/710571385', 
            'way/375314249', 'way/338043682', 'way/46340036', 'way/710571377', 'way/5671777']

drop_ped_index = [cp_pedestrian_osm[cp_pedestrian_osm.get('id') == ped].index[0] 
                  for ped in drop_ped]

allpedestrianosm = cp_pedestrian_osm.drop(index=drop_ped_index)


# allspcenterosm

drop_spcenter = ['node/368062558', 'node/2724236668', 'node/5607297102']

drop_spcenter_index = [cp_sportscenter_osm[cp_sportscenter_osm.get('id') 
                                           == spcenter].index[0] 
                       for spcenter in drop_spcenter]

allspcenterosm = cp_sportscenter_osm.drop(index=drop_spcenter_index)

In [8]:
### map all features with legend and tag

# allwaterosm
allwaterosm_keep = ['id', 'geometry', 'name', 'natural', 'waterway']
centralparkwater = allwaterosm[allwaterosm_keep]
feature_type = centralparkwater['natural'].fillna(centralparkwater['waterway'])
centralparkwater = centralparkwater.assign(feature_type = feature_type).drop(columns = ['natural', 'waterway'])
#centralparkwater = centralparkwater[centralparkwater.get('feature_type') == 'water']

# allpavedosm
allpavedosm_keep = ['id', 'geometry', 'name', 'highway', 'route']
centralparkpaved = allpavedosm[allpavedosm_keep].reset_index().drop(columns='index')
centralparkpaved = centralparkpaved.assign(feature_type = 'paved').drop(columns = ['highway', 'route'])

# allgardenosm
allgardenosm_keep = ['id', 'geometry', 'name', 'leisure']
centralparkgarden = allgardenosm[allgardenosm_keep]
centralparkgarden = centralparkgarden.assign(feature_type = 
                                             centralparkgarden.get('leisure')).drop(columns = 'leisure')

# allbaseballosm
allbaseball_keep = ['id', 'geometry', 'name', 'leisure', 'sport']
centralparkbaseball = allbaseballosm[allbaseball_keep]
centralparkbaseball = centralparkbaseball.assign(feature_type = 
                                                 'pitch: baseball').drop(columns = ['leisure', 'sport'])

# allwoodsosm
allwoods_keep = ['id', 'geometry', 'name', 'natural']
centralparkwoods = allwoodsosm[allwoods_keep]
centralparkwoods = centralparkwoods.assign(feature_type = 
                                           centralparkwoods.get('natural')).drop(columns = 'natural')

# allfieldsosm
allfields_keep = ['id', 'geometry', 'name', 'landuse']
centralparkfields = allfieldosm[allfields_keep]
centralparkfields = centralparkfields.assign(feature_type = 'grass').drop(columns = 'landuse')

# allplaygosm
allplayg_keep = ['id', 'geometry', 'name', 'leisure']
centralparkplayg = allplaygosm[allplayg_keep]
centralparkplayg = centralparkplayg.assign(feature_type = 'playground').drop(columns = 'leisure')

# allbuildingosm
allbuild_keep = ['id', 'geometry', 'name', 'building']
centralparkbuilding = allbuildingosm[allbuild_keep]
centralparkbuilding = centralparkbuilding.assign(feature_type = 'building').drop(columns = 'building')

# allspcenterosm
allspcenter_keep = ['id', 'geometry', 'name', 'leisure']
centralparkspcenter = allspcenterosm[allspcenter_keep]
centralparkspcenter = centralparkspcenter.assign(feature_type = 'sports center').drop(columns = 'leisure')

#allpedestrianosm
allped_keep = ['id', 'geometry', 'name', 'highway']
centralparkpedestrian = allpedestrianosm[allped_keep]
centralparkpedestrian = centralparkpedestrian.assign(feature_type = 'pedestrian').drop(columns = 'highway')

# allbarerockosm
allrock_keep = ['id', 'geometry', 'name', 'natural']
centralparkrock = allbarerockosm[allrock_keep]
centralparkrock = centralparkrock.assign(feature_type = 'bare_rock').drop(columns = 'natural')

# alltoiletsosm
alltoilet_keep = ['id', 'geometry', 'name', 'amenity']
centralparktoilet = alltoiletsosm[alltoilet_keep]
centralparktoilet = centralparktoilet.assign(feature_type = 'toilet').drop(columns = 'amenity')

centralparkfeatures = pd.concat([centralparkgarden, centralparkwater, centralparkfields, 
                                 centralparkwoods, centralparkbaseball, centralparkplayg,
                                 centralparkpaved, centralparkbuilding, centralparkspcenter,
                                 centralparkpedestrian, centralparkrock, centralparktoilet])

centralparkfeaturescmap = ['#6b6865', '#ad6f03', '#ff8cec', '#86b35d', '#000000', '#000000', '#c4a782', '#d96866',
                           '#960906', '#1795e8', '#737373', '#1795e8', '#098f57']

centralparkallfeaturesmap = centralparkfeatures.explore(column='feature_type',
                                                     tooltip=['feature_type', 'name'],
                                                     popup=True,
                                                     tiles="cartodbpositron",
                                                     cmap = centralparkfeaturescmap,
                                                     max_zoom = 10)

centralparkallfeaturesmap

In [10]:
# map only significant features

cpfeatures_short = pd.concat([centralparkgarden, centralparkwater, centralparkfields, 
                              centralparkwoods, centralparkbuilding, centralparkpedestrian])

cpfeatures_shortcmap = ['#ad6f03', '#ff8cec', '#86b35d', '#000000', '#1795e8', 
                        '#1795e8', '#098f57']

cpfeatures_shortmap = cpfeatures_short.explore(column='feature_type',
                                               tooltip=['feature_type', 'name'],
                                               popup=True,
                                               tiles="cartodbpositron",
                                               cmap=cpfeatures_shortcmap)

cpfeatures_shortmap

In [23]:
### remove squirrels outside central park

centralpark_perimeter = centralparkpaved[(centralparkpaved.get('name') == '5th Avenue') |
                                         (centralparkpaved.get('name') == 'Central Park North') |
                                         (centralparkpaved.get('name') == 'Central Park West') |
                                         (centralparkpaved.get('name') == 'Central Park South') |
                                         (centralparkpaved.get('name') == 'Grand Army Plaza') |
                                         (centralparkpaved.get('name') == 'Doris C. Freedman Place') |
                                         (centralparkpaved.get('name') == 'Duke Ellington Circle')]

centralparkperimeter = centralpark_perimeter.unary_union.convex_hull

squirrelinside = nyc_gdf2.within(centralparkperimeter)

nyc_gdf5 = nyc_gdf2.assign(ingarden = squirrelingarden,
                           infields = squirrelinfields,
                           inpedestrian = squirrelinpedestrian,
                           inwoods = squirrelinwoods,
                           inbuilding = squirrelinbuilding,
                           inwater = squirrelinwater,
                           inside = squirrelinside)

nyc_gdf5 = nyc_gdf5[nyc_gdf5.get('inside') == True]
nyc_gdf5

,long,lat,unique_squirrel_id,hectare,shift,date,hectare_squirrel_number,age,primary_fur_color,combination_of_primary_and_highlight_color,...,city_council_districts,police_precincts,geometry,ingarden,infields,inpedestrian,inwoods,inbuilding,inwater,inside
0,-73.957044,40.794851,37E-PM-1006-03,37E,PM,10062018,3,Adult,Gray,Gray+Cinnamon,...,19,13,POINT (-73.95704 40.79485),False,False,False,True,False,False,True
1,-73.976831,40.766718,2E-AM-1010-03,02E,AM,10102018,3,Adult,Cinnamon,Cinnamon+,...,19,13,POINT (-73.97683 40.76672),False,False,False,False,False,False,True
2,-73.975725,40.769703,5D-PM-1018-05,05D,PM,10182018,5,Juvenile,Gray,Gray+,...,19,13,POINT (-73.97572 40.76970),False,False,False,False,False,False,True
3,-73.956570,40.790256,33H-AM-1019-02,33H,AM,10192018,2,Juvenile,Gray,Gray+Cinnamon,...,19,13,POINT (-73.95657 40.79026),False,True,False,False,False,False,True
4,-73.971974,40.769305,6G-PM-1020-02,06G,PM,10202018,2,Adult,Gray,Gray+,...,19,13,POINT (-73.97197 40.76930),False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2963,-73.963943,40.790868,30B-AM-1007-04,30B,AM,10072018,4,Adult,Gray,Gray+,...,19,13,POINT (-73.96394 40.79087),False,False,False,False,False,False,True
2964,-73.970402,40.782560,19A-PM-1013-05,19A,PM,10132018,5,Adult,Gray,Gray+White,...,19,13,POINT (-73.97040 40.78256),False,False,False,False,False,False,True
2965,-73.966587,40.783678,22D-PM-1012-07,22D,PM,10122018,7,Adult,Gray,"Gray+Black, Cinnamon, White",...,19,13,POINT (-73.96659 40.78368),False,False,False,True,False,False,True
2966,-73.963994,40.789915,29B-PM-1010-02,29B,PM,10102018,2,,Gray,"Gray+Cinnamon, White",...,19,13,POINT (-73.96399 40.78992),False,False,False,False,False,False,True


## inside - squirrel *in* feature (eda)

In [24]:
# centralparkbuilding
geom_cpbuild_union = centralparkbuilding.geometry.unary_union
squirrelinbuilding = nyc_gdf2.within(geom_cpbuild_union)

# centralparkgarden
geom_cpgarden_union = centralparkgarden.geometry.unary_union
squirrelingarden = nyc_gdf2.within(geom_cpgarden_union)

# centralparkgrass
geom_cpfields_union = centralparkfields.geometry.unary_union
squirrelinfields = nyc_gdf2.within(geom_cpfields_union)

# centralparkpedestrian
geom_cppedestrian_union = centralparkpedestrian.geometry.unary_union
squirrelinpedestrian = nyc_gdf2.within(geom_cppedestrian_union)

# centralparkwater
geom_cpwater_union = centralparkwater.geometry.unary_union
squirrelinwater = nyc_gdf2.within(geom_cpwater_union)

# centralparkwood
geom_cpwoods_union = centralparkwoods.geometry.unary_union
squirrelinwoods = nyc_gdf2.within(geom_cpwoods_union)

## buffers - squirrel *near* feature

In [117]:
### xx unit radius buffers

# project to a projected CRS for accuracy (instead of a geographic CRS)
nyc_gdf2_2263 = nyc_gdf2.to_crs('epsg:2263')
buffer_radius = 180

# building buffer
centralparkbuilding_2263 = centralparkbuilding.to_crs('epsg: 2263')
building_buffer = centralparkbuilding_2263.geometry.buffer(buffer_radius).unary_union
squirrelinbuildingbuffer = nyc_gdf2_2263.within(building_buffer)

# garden buffer
centralparkgarden_2263 = centralparkgarden.to_crs('epsg: 2263')
garden_buffer = centralparkgarden_2263.geometry.buffer(buffer_radius).unary_union
squirrelingardenbuffer = nyc_gdf2_2263.within(garden_buffer)

# grass buffer
centralparkgrass_2263 = centralparkfields.to_crs('epsg: 2263')
grass_buffer = centralparkgrass_2263.geometry.buffer(buffer_radius).unary_union
squirrelingrassbuffer = nyc_gdf2_2263.within(grass_buffer)

# pedestrian buffer
centralparkpedestrian_2263 = centralparkpedestrian.to_crs('epsg: 2263')
pedestrian_buffer = centralparkpedestrian_2263.geometry.buffer(buffer_radius).unary_union
squirrelinpedestrianbuffer = nyc_gdf2_2263.within(pedestrian_buffer)

# water buffer
centralparkwater_2263 = centralparkwater.to_crs('epsg:2263')
water_buffer = centralparkwater_2263.geometry.buffer(buffer_radius).unary_union
squirrelinwaterbuffer = nyc_gdf2_2263.within(water_buffer)

# woods buffer
centralparkwoods_2263 = centralparkwoods.to_crs('epsg: 2263')
woods_buffer = centralparkwoods_2263.geometry.buffer(buffer_radius).unary_union
squirrelinwoodsbuffer = nyc_gdf2_2263.within(woods_buffer)


nyc_gdf5 = nyc_gdf5.assign(nearbuilding = squirrelinbuildingbuffer,
                           neargarden = squirrelingardenbuffer,
                           neargrass = squirrelingrassbuffer,
                           nearpedestrian = squirrelinpedestrianbuffer,
                           nearwater = squirrelinwaterbuffer,
                           nearwoods = squirrelinwoodsbuffer)

# find size of each
squirrelnearbuilding = nyc_gdf5[nyc_gdf5.get('nearbuilding') == True]
squirrelneargarden = nyc_gdf5[nyc_gdf5.get('neargarden') == True]
squirrelneargrass = nyc_gdf5[nyc_gdf5.get('neargrass') == True]
squirrelnearpedestrian = nyc_gdf5[nyc_gdf5.get('nearpedestrian') == True]
squirrelnearwater = nyc_gdf5[nyc_gdf5.get('nearwater') == True]
squirrelnearwoods = nyc_gdf5[nyc_gdf5.get('nearwoods') == True]

#folium.LayerControl().add_to(cpfeatures_shortmap)
#squirrelnearwater.explore(m = cpfeatures_shortmap)
#squirrelnearwater.explore()

## buffer size comparison dataframe

In [63]:
onerow = pd.DataFrame([[buffer_radius, squirrelnearbuilding.shape[0], squirrelneargarden.shape[0], 
                        squirrelneargrass.shape[0], squirrelnearpedestrian.shape[0], 
                        squirrelnearwater.shape[0], squirrelnearwoods.shape[0]]], 
                      columns = ['buffer_radius', 'building', 
                                         'garden', 'grass', 'pedestrian', 'water', 'woods'])

buffersizecomp = pd.concat([buffersizecomp, onerow])
buffersizecomp

,buffer_radius,building,garden,grass,pedestrian,water,woods
0,100,228,154,544,169,401,640
0,120,320,166,610,212,475,676
0,140,384,195,697,265,562,717
0,160,455,210,760,324,650,758
0,180,552,224,842,360,731,786
0,200,634,243,911,415,810,819


In [67]:
buffersizecomp.reset_index().drop(columns = 'index')

,buffer_radius,building,garden,grass,pedestrian,water,woods
0,100,228,154,544,169,401,640
1,120,320,166,610,212,475,676
2,140,384,195,697,265,562,717
3,160,455,210,760,324,650,758
4,180,552,224,842,360,731,786
5,200,634,243,911,415,810,819


In [122]:
import os

buffersizecomp.to_csv('bufferradiuscomparison/buffersizecompdf.csv')

## folium map

In [118]:
### projections

buffercrs = pyproj.CRS('epsg: 2263')
regcrs = pyproj.CRS('epsg:4326')
project = pyproj.Transformer.from_crs(buffercrs, regcrs, always_xy = True).transform

buildingbufferwgs84 = transform(project, building_buffer)
gardenbufferwgs84 = transform(project, garden_buffer)
grassbufferwgs84 = transform(project, grass_buffer)
pedestrianbufferwgs84 = transform(project, pedestrian_buffer)
waterbufferwgs84 = transform(project, water_buffer)
woodsbufferwgs84 = transform(project, woods_buffer)

In [119]:
buildingbufferwgs84df = gpd.GeoDataFrame(buildingbufferwgs84)
buildingbufferwgs84df = buildingbufferwgs84df.assign(geometry = 
                                               buildingbufferwgs84df.get(0)).drop(columns=0).set_crs('epsg:4326')

gardenbufferwgs84df = gpd.GeoDataFrame(gardenbufferwgs84)
gardenbufferwgs84df = gardenbufferwgs84df.assign(geometry = 
                                               gardenbufferwgs84df.get(0)).drop(columns=0).set_crs('epsg:4326')

grassbufferwgs84df = gpd.GeoDataFrame(grassbufferwgs84)
grassbufferwgs84df = grassbufferwgs84df.assign(geometry = 
                                               grassbufferwgs84df.get(0)).drop(columns=0).set_crs('epsg:4326')

pedestrianbufferwgs84df = gpd.GeoDataFrame(pedestrianbufferwgs84)
pedestrianbufferwgs84df = pedestrianbufferwgs84df.assign(geometry = 
                                               pedestrianbufferwgs84df.get(0)).drop(columns=0).set_crs('epsg:4326')

waterbufferwgs84df = gpd.GeoDataFrame(waterbufferwgs84)
waterbufferwgs84df = waterbufferwgs84df.assign(geometry = 
                                               waterbufferwgs84df.get(0)).drop(columns=0).set_crs('epsg:4326')

woodsbufferwgs84df = gpd.GeoDataFrame(woodsbufferwgs84)
woodsbufferwgs84df = woodsbufferwgs84df.assign(geometry = 
                                               woodsbufferwgs84df.get(0)).drop(columns=0).set_crs('epsg:4326')

In [120]:
cpimap = folium.Map(location = [40.7835, -73.96600], zoom_start = 14, tiles = 'cartodbpositron')

style_functions = lambda x: {'stroke': False, 
                             'fillOpacity': 1.0,
                             'fillColor': '#098f57' if x['properties']['feature_type'] == 'wood' 
                             else '#1795e8' if x['properties']['feature_type'] == 'water'
                             else '#86b35d' if x['properties']['feature_type'] == 'grass'
                             else '#ad6f03' if x['properties']['feature_type'] == 'building'
                             else '#ff8cec' if x['properties']['feature_type'] == 'garden'
                             else '#000000'}

# add individual feature layers
folium.GeoJson(centralparkwater, name='water', 
               style_function=lambda x: {'fillColor': '#1795e8', 'fillOpacity': 1.0, 'stroke':False}).add_to(cpimap)
folium.GeoJson(centralparkwoods, name='woods',
               style_function=lambda x: {'fillColor': '#098f57', 'fillOpacity': 1.0, 'stroke':False}).add_to(cpimap)
folium.GeoJson(centralparkfields, name='grass', 
               style_function=lambda x: {'fillColor': '#86b35d', 'fillOpacity': 1.0, 'stroke':False}).add_to(cpimap)
folium.GeoJson(centralparkgarden, name='garden',
               style_function=lambda x: {'fillColor': '#ff8cec', 'fillOpacity': 1.0, 'stroke':False}).add_to(cpimap)
folium.GeoJson(centralparkbuilding, name='building',
               style_function=lambda x: {'fillColor': '#ad6f03', 'fillOpacity': 1.0, 'stroke':False}).add_to(cpimap)
folium.GeoJson(centralparkpedestrian, name='pedestrain',
               style_function=lambda x: {'fillColor': '#000000', 'fillOpacity': 1.0, 'stroke':False}).add_to(cpimap)

# add buffers
folium.GeoJson(buildingbufferwgs84df, name = 'building buffer', 
               style_function=lambda x: {'fillColor': '#ad6f03', 'fillOpacity': 0.2, 'color': '#ad6f03'}).add_to(cpimap)
folium.GeoJson(gardenbufferwgs84df, name = 'garden buffer',
               style_function=lambda x: {'fillColor': '#ff8cec', 'fillOpacity': 0.2, 'color': '#ff8cec'}).add_to(cpimap)
folium.GeoJson(grassbufferwgs84df, name='grass buffer',
               style_function=lambda x: {'fillColor': '#86b35d', 'fillOpacity': 0.2, 'color': '#86b35d'}).add_to(cpimap)
folium.GeoJson(pedestrianbufferwgs84df, name = 'pedestrian buffer', 
               style_function=lambda x: {'fillColor': '#000000', 'fillOpacity': 0.2, 'color': '#000000'}).add_to(cpimap)
folium.GeoJson(waterbufferwgs84df, name='water buffer', 
               style_function=lambda x: {'fillColor': '#1795e8', 'fillOpacity': 0.2, 'color': '#1795e8'}).add_to(cpimap)
folium.GeoJson(woodsbufferwgs84df, name='woods buffer',
               style_function=lambda x: {'fillColor': '#098f57', 'fillOpacity': 0.2, 'color': '#098f57'}).add_to(cpimap)


folium.LayerControl().add_to(cpimap)

cpimap

In [121]:
cpimap.save('cpimap180ftbuffer.html')